In [ ]:
### 1. Mount Google Drive ###

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
### 2. Define root directory ###

ROOT_DIR = '/content/gdrive/My Drive/yolo v8'

In [ ]:
### 3. Install Ultralytics ###

!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.9 MB/s eta 0:00:00


In [ ]:
### 4. Train model ###

import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=250)  # train the model



Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/gdrive/My Drive/yolo v8/google_colab_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

In [ ]:
### 5. Copy results ###

!scp -r /content/runs '/content/gdrive/My Drive/yolo v8'

In [ ]:
from ultralytics import YOLO

import numpy as np


model = YOLO('/content/gdrive/My Drive/yolo v8/runs/detect/train/weights/best.pt')  # load a custom model

#results = model('/content/gdrive/My Drive/yolo v8/apple.jpg')
# predict on an empty image
#results = model('/content/gdrive/My Drive/yolo v8/pothole.jpg')
# predict on pothole
results = model('/content/gdrive/My Drive/yolo v8/real_waterlog.jpg')
# predict on waterlog
names_dict = model.names  # model.names gives class mapping
# Extract detectionsm
boxes = results[0].boxes  # all bounding boxes
class_ids = boxes.cls.cpu().numpy().astype(int)  # class indices
confidences = boxes.conf.cpu().numpy()          # confidence scores

print("Class IDs:", class_ids)
print("Confidences:", confidences)

# Get most confident prediction
best_idx = np.argmax(confidences)
if(max(confidences)<0.8):
    print("nothing found")
else :
    print("Predicted class:", names_dict[class_ids[best_idx]])
    print("Confidence:", confidences[best_idx])


image 1/1 /content/gdrive/My Drive/yolo v8/waterlog.jpg: 480x640 1 waterlog, 221.2ms
Speed: 3.8ms preprocess, 221.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)
Class IDs: [1]
Confidences: [    0.88352]
Predicted class: waterlog
Confidence: 0.8835217


In [ ]:
w# Install dependencies (only needed in Colab)
!pip install ultralytics opencv-python

import os
import cv2
from ultralytics import YOLO
from IPython.display import display, HTML

# Set paths (adjust these paths based on your file locations)
VIDEOS_DIR = '/content/gdrive/My Drive/yolo v8'  # You can upload your video to this folder
os.makedirs(VIDEOS_DIR, exist_ok=True)

video_path = os.path.join(VIDEOS_DIR, 'potholes.mp4')  # upload video manually or from Drive
video_path_out = f'{video_path}_out.mp4'

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    raise ValueError("Error reading the video file. Please check the path or upload a valid video.")

H, W, _ = frame.shape
out = cv2.VideoWriter(video_path_out, cv2.VideoWriter_fourcc(*'MP4V'),
                      int(cap.get(cv2.CAP_PROP_FPS)), (W, H))

model = YOLO('/content/gdrive/My Drive/yolo v8/runs/detect/train/weights/best.pt')

threshold = 0.7

while ret:
    results = model(frame)[0]

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            cv2.putText(frame, results.names[int(class_id)].upper(),
                        (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    out.write(frame)
    ret, frame = cap.read()

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Processed video saved at: {video_path_out}")

# Display output video in Colab
from IPython.display import Video
Video(video_path_out, embed=True, width=640, height=480)
